In [1]:
!pip install pandas
!pip install torch
!pip install torchvision
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install scikit-learn
!pip install tensorboard


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/7_/04ycmw7s02j2kfzppg4hj70h0000gn/T/pip-req-build-xu13_eu4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/7_/04ycmw7s02j2kfzppg4hj70h0000gn/T/pip-req-build-xu13_eu4
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of

In [2]:
import csv
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from pathlib import Path
from tqdm.auto import trange

from classes.adaptation import calc_loss, construct_model
from classes.asymmetricRecall import AsymmetricRecall
from classes.embDataset import EmbDataset

from utils.adaptationPreprocess import create_paired_embeddings_dict, merge_dicts_with_csv
from utils.train_functions import train, validate, split_train_test

/Users/hec/Documents/unimelb/computer-vision/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LEFT_FILES_PATH = 'data/train/left/*'
RIGHT_FILES_PATH = 'data/train/right/*'
ALL_FILES_PATH = 'data/train/all/*'
ALL_ENCODINGS_PATH = 'data/train_all_encodings.npy'
ALL_FULL_ENCODINGS_PATH = 'data/train_all_full_encodings.npy'
CSV_FILENAME = 'data/train.csv'
SAVE_TO = 'output/'

In [4]:
test = np.load('data/clip_rn50_with_pca_256.npy', allow_pickle=True).reshape(-1)[0]
print(test['left'].shape)
print(test['right'].shape)
print(type(test['left_name']))
print(type(test), type(test['left']), type(test['right']))

(8398, 256)
(8398, 256)
<class 'list'>
<class 'dict'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [5]:
is_cuda = torch.cuda.is_available()
if not is_cuda:
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

In [6]:
runs = 1
temperature = 1
topk = [2]
epochs = 100
test_split = 0.7
model_name = 'original'

In [7]:
all_encodings = np.load(ALL_ENCODINGS_PATH)
all_full_encodings = np.load(ALL_FULL_ENCODINGS_PATH)
# Number of dimensions of the encodings (output of 1 model in the first place, then output of the PCA of the concatenation of many layers)
dimensions = all_encodings.shape[1]
print(all_encodings.shape)
print(all_full_encodings.shape)

(4000, 640)
(4000, 6080)


In [8]:
paired_embeddings = create_paired_embeddings_dict(all_full_encodings, ALL_FILES_PATH, LEFT_FILES_PATH, RIGHT_FILES_PATH)

In [9]:
merged_dicts = merge_dicts_with_csv(CSV_FILENAME, paired_embeddings)
print(merged_dicts['left'].shape)

(2000, 6080)


In [10]:
model = construct_model(model_name, dimensions).to(device)

In [11]:
# load the dataset and split

embds_dataset = EmbDataset(merged_dicts, only_original=False)

print("total ", len(embds_dataset))
in_dim = embds_dataset[0][0]["left"].shape[0]


splitted = split_train_test(embds_dataset, test_split, device)
train_set, valid_set = splitted["train"], splitted["test"]

print("train set without aug size ", len(train_set[0]["left"]))
print("train set with    aug size ", len(train_set[1]["left"]))
print("test  set without aug size  ", len(valid_set["left"]))

metrics_of_all_runs = []

info = (
    "Embd path: "
    + str(ALL_FULL_ENCODINGS_PATH)
    + "\n"
    + "Using augmented images: "
    + str(embds_dataset.augmented)
    + "\n"
)

total  2000
train set without aug size  300
train set with    aug size  300
test  set without aug size   700


In [12]:
for run in trange(runs):
    run_path = Path(SAVE_TO) / f"run{run+1}"
    model = construct_model(model_name, in_dim).to(device)

    writer = SummaryWriter(run_path)
    writer.add_text("Temperature: ", str(temperature))
    writer.add_text("Model", str(model).replace("\n", "  \n"))
    writer.add_text("Informations: ", info.replace("\n", "  \n"))

    if model != "dummy":
        optimizer = torch.optim.Adam(model.parameters())
        # writer.add_text("Optimizer", str(optimizer).replace("\n", "  \n"))

        metrics_per_run = train(
            model,
            optimizer,
            calc_loss,
            train_set,
            valid_set,
            epochs,
            run_path,
            temperature,
            topk,
            device,
        )
        # save the model and the optimizer state_dics for this run
        torch.save(
            {
                "epoch": epochs,
                "state_dict": model.state_dict(),
                "optimzier": optimizer.state_dict(),
            },
            run_path / "model_and_optimizer.pt",
        )
    else:
        metrics_per_run = validate(
            model,
            calc_loss,
            valid_set,
            temperature,
            topk,
            device,
        )
        for metric, val in metrics_per_run.items():
            writer.add_scalar(metric, val, 1)
            writer.flush()
            

    np.save(run_path / "data", metrics_per_run)

    metrics_of_all_runs.append(metrics_per_run)

    splitted = split_train_test(embds_dataset, test_split, device)
    train_set, valid_set = splitted["train"], splitted["test"]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:59<00:00, 59.88s/it]


In [13]:

# save all runs metrics into one file
np.save(Path(SAVE_TO) / "all_runs", metrics_of_all_runs)

avg_path = Path(SAVE_TO) / "avg"
writer = SummaryWriter(avg_path)

In [14]:
test = np.load('output/run1/data.npy', allow_pickle=True)
print(test)

{'train': {'Train/Loss': [5.703790187835693, 5.703559875488281, 5.7035813331604, 5.703587532043457, 5.70358943939209, 5.703582763671875, 5.703573226928711, 5.703558921813965, 5.703542709350586, 5.703524112701416, 5.703502655029297, 5.703476428985596, 5.703447341918945, 5.7034125328063965, 5.703370571136475, 5.7033233642578125, 5.703265190124512, 5.703198432922363, 5.7031097412109375, 5.703008651733398, 5.702878952026367, 5.702719211578369, 5.702506065368652, 5.7022294998168945, 5.701840400695801, 5.70127534866333, 5.7003960609436035, 5.698924541473389, 5.69627571105957, 5.690690994262695, 5.67966365814209, 5.660128116607666, 5.634043216705322, 5.620916366577148, 5.602248191833496, 5.586994171142578, 5.584691047668457, 5.585503101348877, 5.551000118255615, 5.548064231872559, 5.576379776000977, 5.574302673339844, 5.555549144744873, 5.546905994415283, 5.491826057434082, 5.51329231262207, 5.527447700500488, 5.536275863647461, 5.494196891784668, 5.522518634796143, 5.540899753570557, 5.56414